# Notebook que permite resolvel el problema del Knapsack con valores asociados

#### Esta versión incluye:
    Optimización sparse
    


In [1]:
import numpy as np
from scipy.sparse import csr_matrix
import torch
from time import time
from scipy.sparse import diags
import sys

### Generación de los tensores

In [2]:

def tensor_initial_generator(peso: int, valor: int, n_elementos: int, capacidad: int,previous_weight_solution: int) -> csr_matrix:
    """
    Generates the initial tensor in sparse matrix format.

    Parameters:
    - peso (int): Weight of the element.
    - valor (int): Value associated with the element.
    - n_elementos (int): Number of elements available.
    - capacidad (int): Capacity limit.
    - previous_weight_solution (int): Previous weight contribution to the solution.


    Returns:
    - csr_matrix: Sparse representation of the tensor matrix with calculated values.
    """
    # Calculate matrix dimensions
    tamaño_1 = min((capacidad - previous_weight_solution) // peso, n_elementos) + 1
    tamaño_2 = min(capacidad, (tamaño_1 - 1) * peso + previous_weight_solution) + 1

    # Initialize sparse matrix in CSR format
    tensor_data = []
    tensor_rows = []
    tensor_cols = []

    for i in range(tamaño_1):
        elemento = i * peso + previous_weight_solution
        if elemento < tamaño_2:
            tensor_data.append(valor ** i)
            tensor_rows.append(i)
            tensor_cols.append(elemento)

    # Create sparse matrix directly with data, row, and column indices
    tensor = csr_matrix((tensor_data, (tensor_rows, tensor_cols)), shape=(tamaño_1, tamaño_2))

    return tensor

def tensor_intermediate_generator(peso: int, valor: int, n_elementos: int, capacidad: int, previous_weight: int) -> csr_matrix:
    tamaño_1 = previous_weight
    tamaño_2 = min(capacidad, n_elementos * peso + previous_weight - 1) + 1
    diagonals = [1]
    for k in range(1,n_elementos+1):
        diagonals.append(valor**k)
    positions = [0]
    for k in range(1,n_elementos+1):
        positions.append(peso*k)
    tensor = diags(diagonals, positions, shape=(tamaño_1,tamaño_2))
    return tensor


def tensor_final_generator(peso:int, valor:int,n_elementos: int, capacidad: int, previous_weight:int)->np.array:
    tamaño_1 = previous_weight
    tensor = np.zeros((tamaño_1))
    for i in range(tamaño_1):
        n_elementos_posibles = min((capacidad -i)//peso,n_elementos)+1
        for j in range(n_elementos_posibles):
            tensor[i] += valor**j#*np.exp(1j*2*np.pi*j/n_elementos_posibles).astype(complex128)
    tensor = np.ones((tamaño_1))
    return csr_matrix(tensor)

In [3]:
def tensor_generator(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int):
    
    lista_de_tensores = []
    n_clases = len(pesos)

    tensor = tensor_initial_generator(pesos[0],valores[0],n_elementos[0],capacidad,0)
    lista_de_tensores.append(tensor)

    for n in range(1, n_clases-1):
        tensor = tensor_intermediate_generator(pesos[n], valores[n], n_elementos[n], capacidad, tensor.shape[1])
        lista_de_tensores.append(tensor)

    tensor = tensor_final_generator(pesos[-1], valores[-1], n_elementos[-1], capacidad, tensor.shape[1])

    lista_de_tensores.append(tensor)
 
    return lista_de_tensores

### Función de contracción de los tensores

In [4]:
def tensor_contraction(lista_de_tensores):
    n_tensores = len(lista_de_tensores)
    tensores_intermedios = []

    # Iniciar el vector como el último tensor en la lista, convertido temporalmente a PyTorch en la GPU
    vector = torch.tensor(lista_de_tensores[-1].toarray(), device='cuda')
    tensores_intermedios.append(csr_matrix(vector.cpu().numpy()))  # Guardamos en formato sparse en CPU

    # Realizar la contracción de tensores uno por uno
    for j in range(n_tensores - 1, 0, -1):
        # Convertir el tensor actual a formato denso de PyTorch temporalmente
        current_tensor = torch.tensor(lista_de_tensores[j - 1].toarray(), device='cuda')
        
        # Realizar la multiplicación y almacenar el resultado temporalmente en GPU
        #vector = vector.to(torch.complex128)
        if j == n_tensores - 1:
            
            vector = current_tensor @ vector.T
        else:
            vector = current_tensor @ vector
        
        # Normalización si es necesario para evitar overflow
        if torch.max(vector) > 1e200:
           vector = vector * 1e-15
        
        # Guardar el resultado intermedio como un tensor disperso en CPU
        tensores_intermedios.append(csr_matrix(vector.cpu().numpy()))
    
    # Convertir la lista de tensores intermedios de nuevo a formato CPU y retornar el último vector en CPU
    vector = csr_matrix(vector.cpu().numpy())
    tensores_intermedios.reverse()
    
    return vector, tensores_intermedios

### Función general

In [5]:

def solver(
    pesos: np.array, 
    valores: np.array, 
    n_elementos: np.array, 
    capacidad: int, 
    tao: float
) -> tuple:
    """
    Solves the knapsack problem without explicit values using tensor-based dynamic programming.

    Parameters:
    - pesos (np.array): Array of weights for each item class.
    - valores (np.array): Array of values for each item class.
    - n_elementos (np.array): Array of item counts available for each class.
    - capacidad (int): Capacity limit of the knapsack.
    - tao (float): Scaling factor for exponential transformation of values.

    Returns:
    - tuple: Total weight, total value, and maximum value achieved by the solution.
    """
   
    valores_scaled = np.exp(valores * tao)
    n_clases = len(pesos)
    solution = np.zeros(n_clases)
    lista_de_tensores = tensor_generator(pesos, valores_scaled, n_elementos, capacidad)

    # Compute output vector and intermediate tensors through contraction
    vector_salida, tensores_intermedios = tensor_contraction(lista_de_tensores)

        
    # Determine the maximum value and initialize the solution vector
    max_value = np.max(tensores_intermedios[0])
 
    solution[0] = np.argmax(abs(vector_salida))

    
    # Iterate over classes to build the solution progressively
    for n in range(1, n_clases - 1):

        cumulative_weight = int(np.dot(solution[:n], pesos[:n]))
    
        new_initial_tensor = tensor_initial_generator(
            pesos[n], valores_scaled[n], n_elementos[n], capacidad, cumulative_weight
        )

        if n == n_clases - 2:
            tensores_intermedios[n + 1] = tensores_intermedios[n + 1].T  # Transpose for final calculation

        solution[n] = np.argmax(abs(new_initial_tensor @ tensores_intermedios[n + 1][:new_initial_tensor.shape[1]]))
   


    cumulative_weight = np.dot(solution[:-1], pesos[:-1])
    solution[-1] = min((capacidad - cumulative_weight) // pesos[-1], n_elementos[-1])


    total_weight = np.dot(solution, pesos)
    total_value = np.dot(solution, valores)
    
    print("El peso total es:", total_weight)
    print("El valor total es:", total_value)
    
    return total_weight, total_value, max_value

### Funciones auxiliares para la realización de las pruebas

In [6]:
def mochila_greedy(pesos, valores, capacidad, n_elementos):
    n_clases = len(pesos)
    
    # Calculamos la relación valor/peso para cada clase
    valor_peso = valores / pesos
    
    # Ordenamos los elementos por su valor/peso de mayor a menor
    indices_ordenados = np.argsort(valor_peso)[::-1]  # Orden descendente
    
    # Inicializamos variables
    peso_actual = 0
    valor_actual = 0
    solucion = np.zeros(n_clases, dtype=int)
    
    # Recorremos los objetos en orden greedy
    for i in indices_ordenados:
        # Tomamos la mayor cantidad posible de este objeto sin exceder la capacidad
        cantidad = min(n_elementos[i], (capacidad - peso_actual) // pesos[i])
        solucion[i] = cantidad
        peso_actual += cantidad * pesos[i]
        valor_actual += cantidad * valores[i]
        
        # Si llenamos la capacidad, salimos del bucle
        if peso_actual >= capacidad:
            break
    
    return solucion, valor_actual


In [9]:

clases = 1000
capacidad = 2000
np.random.seed(1)
pesos = np.random.randint(1,10,clases)
pesos = np.ones(clases,dtype=int)

valores = np.random.rand(clases)
n_elementos = [5]*clases

tao = 1
lambda1 = 1
inicio = time()
b=solver(pesos,valores, n_elementos, capacidad, tao)
fin = time()
print("tiempo de ejecución", fin-inicio)
mejor_solucion, mejor_valor = mochila_greedy(pesos, valores, capacidad, n_elementos)

#print("Mejor solución encontrada (greedy):", mejor_solucion)
print("Valor total de la mejor solución (greedy):", mejor_valor)
print("Peso total de la mejor solución (greedy):", np.sum(mejor_solucion * pesos))

El peso total es: 2000.0
El valor total es: 1626.7411323695917
tiempo de ejecución 5.1527650356292725
Valor total de la mejor solución (greedy): 1631.1344897290558
Peso total de la mejor solución (greedy): 2000
